<a href="https://colab.research.google.com/github/ibitoladgr8/Machine-Learning/blob/main/Sainsbury_task_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
catalogue_disc = pd.read_csv('/content/CatalogueDiscontinuation.csv')
product_details = pd.read_csv('/content/ProductDetails.csv')
data_dictionary = pd.read_excel('/content/DataDictionary.xlsx')

In [ ]:
data_dictionary

,Dataset,Field Seq,Field Name,Data Type,Description
0,ProductDetails,1,ProductKey,Integer (classifier),Unique product identifier
1,ProductDetails,2,Supplier,Integer (classifier),Supplier identifier. A supplier is typically a...
2,ProductDetails,3,HierarchyLevel1,Integer (classifier),Product hierarchy by type of products. Example...
3,ProductDetails,4,HierarchyLevel2,Integer (classifier),HierarchyLevel1 maps many-to-one to HierarchyL...
4,ProductDetails,5,DIorDOM,Text (classifier),Indicates whether a product is sourced domesti...
5,ProductDetails,6,Seasonal,Boolean,"1 for products that have a distinct ""on"" seaso..."
6,CatalogueDiscontinuation,1,CatEdition,Integer (ordinal),"There is a range refresh every 6 months, still..."
7,CatalogueDiscontinuation,2,SpringSummer,Boolean,"Each range is either a spring/summer range, la..."
8,CatalogueDiscontinuation,3,ProductKey,Integer (classifier),Unique product identifier
9,CatalogueDiscontinuation,4,WeeksOut,Integer (ordinal),Number of weeks before the date of potential d...


In [ ]:
product_details

,ProductKey,Supplier,HierarchyLevel1,HierarchyLevel2,DIorDOM,Seasonal
0,30437,13194,80445,28611,DOM,False
1,68587,13194,80445,28611,DOM,False
2,46645,13194,80445,28611,DOM,False
3,49343,13194,80445,28611,DOM,False
4,64223,13194,80445,28611,DOM,False
...,...,...,...,...,...,...
77367,62220,66805,75452,9740,DI,False
77368,53837,66805,75452,9740,DI,False
77369,28091,66805,75452,6840,DI,False
77370,1234,15011,75452,9740,DI,False


In [ ]:
catalogue_disc

,CatEdition,SpringSummer,ProductKey,WeeksOut,Status,SalePriceIncVAT,ForecastPerWeek,ActualsPerWeek,DiscontinuedTF
0,89,True,8108,-11,RI,43.99,250.545,248.642,False
1,89,True,8108,-10,RI,43.99,255.700,244.333,False
2,89,True,8108,-9,RI,43.99,206.111,252.187,False
3,89,True,8108,-8,RI,43.99,188.625,254.764,False
4,89,True,8108,-7,RI,43.99,169.142,249.000,False
...,...,...,...,...,...,...,...,...,...
3625654,89,True,8108,-16,RI,43.99,633.750,0.000,False
3625655,89,True,8108,-15,RI,43.99,564.000,178.200,False
3625656,89,True,8108,-14,RI,43.99,539.214,236.363,False
3625657,89,True,8108,-13,RI,43.99,346.692,251.166,False


In [ ]:
unique_product_keys = catalogue_disc['ProductKey'].nunique()
print(f"Number of unique product keys in catalogue_disc: {unique_product_keys}")

unitque_cat_edition = catalogue_disc['CatEdition'].nunique()
print(f"Number of unique CatEdition in catalogue_disc: {unitque_cat_edition}")

unique_statuse = catalogue_disc['Status'].nunique()
print(f"Number of unique Status in catalogue_disc: {unique_statuse}")

Number of unique product keys in catalogue_disc: 70227
Number of unique CatEdition in catalogue_disc: 7
Number of unique Status in catalogue_disc: 2


In [ ]:
# Group by ProductKey and CatEdition
agg_df = catalogue_disc.groupby(["ProductKey", "CatEdition"]).agg({
    "ActualsPerWeek": ["mean", "sum", "max", "std", "last"],
    "ForecastPerWeek": ["mean", "sum", "max", "std", "last"],
    "SalePriceIncVAT": ["mean", "min", "max", "last"],
    "DiscontinuedTF": "first"    # label (same across weeks)
})

# Flatten column names
agg_df.columns = ["_".join(col).strip() for col in agg_df.columns.values]
agg_df = agg_df.reset_index()

# Add derived feature: Forecast accuracy ratio
agg_df["ForecastAccuracy"] = agg_df["ActualsPerWeek_sum"] / (agg_df["ForecastPerWeek_sum"] + 1e-9)

# Example output
agg_df.head()

,ProductKey,CatEdition,ActualsPerWeek_mean,ActualsPerWeek_sum,ActualsPerWeek_max,ActualsPerWeek_std,ActualsPerWeek_last,ForecastPerWeek_mean,ForecastPerWeek_sum,ForecastPerWeek_max,ForecastPerWeek_std,ForecastPerWeek_last,SalePriceIncVAT_mean,SalePriceIncVAT_min,SalePriceIncVAT_max,SalePriceIncVAT_last,DiscontinuedTF_first,ForecastAccuracy
0,2,94,17.611083,211.333,24.000,3.131604,12.250,20.972750,251.673,27.916,3.557149,16.923,9.00,9.00,9.00,9.00,True,0.839713
1,3,91,22.964250,551.142,45.208,6.603558,45.208,41.445125,994.683,58.666,14.612308,46.000,12.00,12.00,12.00,12.00,True,0.554088
2,4,94,49.535000,594.420,175.500,53.384693,175.500,429.713417,5156.561,560.153,78.912874,560.153,10.00,10.00,10.00,10.00,True,0.115275
3,5,90,0.212500,1.275,0.375,0.177744,0.375,4.936000,29.616,12.833,4.470541,2.000,18.99,18.99,18.99,18.99,True,0.043051
4,5,91,4.252826,97.815,8.000,2.300975,7.458,12.261522,282.015,41.923,12.791203,5.000,18.99,18.99,18.99,18.99,True,0.346843


In [ ]:
agg_df.drop(['SalePriceIncVAT_min','SalePriceIncVAT_max','SalePriceIncVAT_last'], axis=1)

,ProductKey,CatEdition,ActualsPerWeek_mean,ActualsPerWeek_sum,ActualsPerWeek_max,ActualsPerWeek_std,ActualsPerWeek_last,ForecastPerWeek_mean,ForecastPerWeek_sum,ForecastPerWeek_max,ForecastPerWeek_std,ForecastPerWeek_last,SalePriceIncVAT_mean,DiscontinuedTF_first,ForecastAccuracy
0,2,94,17.611083,211.333,24.000,3.131604,12.250,20.972750,251.673,27.916,3.557149,16.923,9.00,True,0.839713
1,3,91,22.964250,551.142,45.208,6.603558,45.208,41.445125,994.683,58.666,14.612308,46.000,12.00,True,0.554088
2,4,94,49.535000,594.420,175.500,53.384693,175.500,429.713417,5156.561,560.153,78.912874,560.153,10.00,True,0.115275
3,5,90,0.212500,1.275,0.375,0.177744,0.375,4.936000,29.616,12.833,4.470541,2.000,18.99,True,0.043051
4,5,91,4.252826,97.815,8.000,2.300975,7.458,12.261522,282.015,41.923,12.791203,5.000,18.99,True,0.346843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178900,80528,94,39.296333,471.556,44.333,3.317979,44.333,36.082667,432.992,61.142,13.117565,53.000,16.00,True,1.089064
178901,80530,90,172.960333,4151.048,231.000,32.418478,141.958,154.594083,3710.258,217.727,35.199664,112.000,19.99,False,1.118803
178902,80530,91,264.281167,6342.748,293.750,22.626268,289.500,300.675500,7216.212,368.500,44.806875,254.000,19.99,True,0.878958
178903,80531,90,0.260200,2.602,1.125,0.404026,1.125,10.037700,100.377,21.875,6.932394,8.000,299.99,False,0.025922


In [ ]:
# Compute number of unique weeks per product
weeks_active_df = (
    catalogue_disc.groupby(["ProductKey", "CatEdition"])
    .agg(weeks_active=("WeeksOut", "nunique"))
    .reset_index()
)

# Merge back into your aggregated dataset
agg_df = agg_df.merge(weeks_active_df, on=["ProductKey", "CatEdition"], how="left")
agg_df.set_index(["ProductKey", "CatEdition"], inplace=True)

In [ ]:
agg_df

,ProductKey,CatEdition,ActualsPerWeek_mean,ActualsPerWeek_sum,ActualsPerWeek_max,ActualsPerWeek_std,ActualsPerWeek_last,ForecastPerWeek_mean,ForecastPerWeek_sum,ForecastPerWeek_max,ForecastPerWeek_std,ForecastPerWeek_last,SalePriceIncVAT_mean,DiscontinuedTF_first,ForecastAccuracy,weeks_active
0,2,94,17.611083,211.333,24.000,3.131604,12.250,20.972750,251.673,27.916,3.557149,16.923,9.00,True,0.839713,12
1,3,91,22.964250,551.142,45.208,6.603558,45.208,41.445125,994.683,58.666,14.612308,46.000,12.00,True,0.554088,24
2,4,94,49.535000,594.420,175.500,53.384693,175.500,429.713417,5156.561,560.153,78.912874,560.153,10.00,True,0.115275,12
3,5,90,0.212500,1.275,0.375,0.177744,0.375,4.936000,29.616,12.833,4.470541,2.000,18.99,True,0.043051,6
4,5,91,4.252826,97.815,8.000,2.300975,7.458,12.261522,282.015,41.923,12.791203,5.000,18.99,True,0.346843,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178900,80528,94,39.296333,471.556,44.333,3.317979,44.333,36.082667,432.992,61.142,13.117565,53.000,16.00,True,1.089064,12
178901,80530,90,172.960333,4151.048,231.000,32.418478,141.958,154.594083,3710.258,217.727,35.199664,112.000,19.99,False,1.118803,24
178902,80530,91,264.281167,6342.748,293.750,22.626268,289.500,300.675500,7216.212,368.500,44.806875,254.000,19.99,True,0.878958,24
178903,80531,90,0.260200,2.602,1.125,0.404026,1.125,10.037700,100.377,21.875,6.932394,8.000,299.99,False,0.025922,10


In [ ]:
# Make sure identifiers are columns
agg_df = agg_df.reset_index()

core_features = [
    "ProductKey", "CatEdition",         # identifiers
    "ActualsPerWeek_sum",               # total demand
    "ActualsPerWeek_std",               # demand volatility
    "ActualsPerWeek_last",              # recent performance
    "ForecastPerWeek_sum",              # total expected demand
    "ForecastPerWeek_std",              # forecast volatility
    "SalePriceIncVAT_mean",             # average price
    "ForecastAccuracy",                 # efficiency
    "weeks_active",                     # lifetime
    "DiscontinuedTF_first"              # target
]

# Select reduced dataset
reduced_df = agg_df[core_features].copy()
reduced_df = reduced_df.rename(columns={"DiscontinuedTF_first": "Discontinued"})

print(reduced_df.head())


   ProductKey  CatEdition  ActualsPerWeek_sum  ActualsPerWeek_std  \
0           2          94             211.333            3.131604   
1           3          91             551.142            6.603558   
2           4          94             594.420           53.384693   
3           5          90               1.275            0.177744   
4           5          91              97.815            2.300975   

   ActualsPerWeek_last  ForecastPerWeek_sum  ForecastPerWeek_std  \
0               12.250              251.673             3.557149   
1               45.208              994.683            14.612308   
2              175.500             5156.561            78.912874   
3                0.375               29.616             4.470541   
4                7.458              282.015            12.791203   

   SalePriceIncVAT_mean  ForecastAccuracy  weeks_active  Discontinued  
0                  9.00          0.839713            12          True  
1                 12.00         

In [ ]:
final_df = pd.merge(
    reduced_df,
    product_details,
    on="ProductKey",
    how="left"   # keep all rows from agg_df
)

# Step 4: Clean up
print("Shape after merge:", final_df.shape)
print("Missing values per column:\n", final_df.isna().sum())

Shape after merge: (178905, 16)
Missing values per column:
 ProductKey                 0
CatEdition                 0
ActualsPerWeek_sum         0
ActualsPerWeek_std      2417
ActualsPerWeek_last        0
ForecastPerWeek_sum        0
ForecastPerWeek_std     2417
SalePriceIncVAT_mean       0
ForecastAccuracy           0
weeks_active               0
Discontinued               0
Supplier                   0
HierarchyLevel1            0
HierarchyLevel2            0
DIorDOM                    0
Seasonal                   0
dtype: int64


In [ ]:
final_df['CatEdition'].unique()

array([94, 91, 90, 92, 93, 88, 89])

In [ ]:
# Map CatEdition values to smaller values
cat_edition_mapping = {edition: i for i, edition in enumerate(final_df['CatEdition'].unique())}
final_df['CatEdition'] = final_df['CatEdition'].map(cat_edition_mapping)

display(final_df.head())

,ProductKey,CatEdition,ActualsPerWeek_sum,ActualsPerWeek_std,ActualsPerWeek_last,ForecastPerWeek_sum,ForecastPerWeek_std,SalePriceIncVAT_mean,ForecastAccuracy,weeks_active,Discontinued,Supplier,HierarchyLevel1,HierarchyLevel2,DIorDOM,Seasonal
0,2,0,211.333,3.131604,12.250,251.673,3.557149,9.00,0.839713,12,True,73968,21640,9740,DI,False
1,3,1,551.142,6.603558,45.208,994.683,14.612308,12.00,0.554088,24,True,13873,21640,6840,DI,False
2,4,0,594.420,53.384693,175.500,5156.561,78.912874,10.00,0.115275,12,True,56450,36227,9740,DI,True
3,5,2,1.275,0.177744,0.375,29.616,4.470541,18.99,0.043051,6,True,43394,32593,28611,DOM,False
4,5,1,97.815,2.300975,7.458,282.015,12.791203,18.99,0.346843,23,True,43394,32593,28611,DOM,False


In [ ]:
final_df["Discontinued"] = final_df["Discontinued"].astype(int)


In [ ]:
final_df = pd.merge(
    final_df,
    catalogue_disc[['ProductKey', 'CatEdition', 'SpringSummer', 'Status']],
    on=['ProductKey', 'CatEdition'],
    how='left'
)

# Drop duplicate columns introduced by merge
final_df = final_df.loc[:,~final_df.columns.duplicated()]

In [ ]:
def get_categorical_columns(df, exclude=[]):
    cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    return list(set(cat_cols) - set(exclude))

def get_numerical_columns(df, exclude=[]):
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    return list(set(num_cols) - set(exclude))


In [ ]:
final_df['CatEdition'].dtype

dtype('int64')

In [ ]:
categorical_cols = get_categorical_columns(final_df, exclude=["ProductKey", "Discontinued"])
numerical_cols = get_numerical_columns(final_df, exclude=["ProductKey", "Discontinued"])
print("Categorical:", categorical_cols)
print("Numerical:", numerical_cols)


Categorical: ['DIorDOM', 'Status', 'SpringSummer', 'Seasonal']
Numerical: ['HierarchyLevel1', 'ForecastPerWeek_std', 'ActualsPerWeek_std', 'ActualsPerWeek_last', 'ActualsPerWeek_sum', 'SalePriceIncVAT_mean', 'ForecastAccuracy', 'HierarchyLevel2', 'weeks_active', 'Supplier', 'CatEdition', 'ForecastPerWeek_sum']


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.8 MB/s eta 0:00:00


In [ ]:
final_df

,ProductKey,CatEdition,ActualsPerWeek_sum,ActualsPerWeek_std,ActualsPerWeek_last,ForecastPerWeek_sum,ForecastPerWeek_std,SalePriceIncVAT_mean,ForecastAccuracy,weeks_active,Discontinued,Supplier,HierarchyLevel1,HierarchyLevel2,DIorDOM,Seasonal,SpringSummer,Status
0,2,94,211.333,3.131604,12.25,251.673,3.557149,9.00,0.839713,12,1,73968,21640,9740,DI,False,False,RI
1,2,94,211.333,3.131604,12.25,251.673,3.557149,9.00,0.839713,12,1,73968,21640,9740,DI,False,False,RI
2,2,94,211.333,3.131604,12.25,251.673,3.557149,9.00,0.839713,12,1,73968,21640,9740,DI,False,False,RI
3,2,94,211.333,3.131604,12.25,251.673,3.557149,9.00,0.839713,12,1,73968,21640,9740,DI,False,False,RI
4,2,94,211.333,3.131604,12.25,251.673,3.557149,9.00,0.839713,12,1,73968,21640,9740,DI,False,False,RI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625654,80531,91,274.192,2.386037,16.75,364.256,9.251729,299.99,0.752745,24,1,76358,51909,18876,DOM,False,True,RO
3625655,80531,91,274.192,2.386037,16.75,364.256,9.251729,299.99,0.752745,24,1,76358,51909,18876,DOM,False,True,RO
3625656,80531,91,274.192,2.386037,16.75,364.256,9.251729,299.99,0.752745,24,1,76358,51909,18876,DOM,False,True,RO
3625657,80531,91,274.192,2.386037,16.75,364.256,9.251729,299.99,0.752745,24,1,76358,51909,18876,DOM,False,True,RO


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier, Pool

# Features & target
X = final_df.drop(columns=['Discontinued', 'ProductKey'])
y = final_df['Discontinued']
groups = final_df['CatEdition']

# Categorical columns
cat_cols = [
    'DIorDOM', 'Status', 'SpringSummer', 'Seasonal',
    'HierarchyLevel1', 'HierarchyLevel2', 'Supplier', 'CatEdition'
]

# GroupKFold setup
gkf = GroupKFold(n_splits=5)

aucs, accs, f1s = [], [], []

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    print(f"\n===== Fold {fold+1} =====")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # CatBoost Pool
    train_pool = Pool(X_train, y_train, cat_features=cat_cols)
    val_pool = Pool(X_val, y_val, cat_features=cat_cols)

    # CatBoost Model
    model = CatBoostClassifier(
        iterations=2000,
        learning_rate=0.02,
        depth=8,
        l2_leaf_reg=5,
        eval_metric="AUC",
        random_seed=42,
        early_stopping_rounds=100,
        verbose=200
    )

    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    # Predict probabilities
    y_pred_proba = model.predict_proba(val_pool)[:, 1]

    # Find best threshold based on F1
    best_thresh, best_f1 = 0.5, 0.0
    for t in np.arange(0.1, 0.9, 0.01):
        preds = (y_pred_proba > t).astype(int)
        f1 = f1_score(y_val, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = t

    # Apply best threshold
    y_pred = (y_pred_proba > best_thresh).astype(int)

    # Metrics
    auc = roc_auc_score(y_val, y_pred_proba)
    acc = accuracy_score(y_val, y_pred)
    f1_val = f1_score(y_val, y_pred)

    print(f"Best threshold = {best_thresh:.2f}")
    print(f"AUC = {auc:.4f}, Accuracy = {acc:.4f}, F1 = {f1_val:.4f}")

    aucs.append(auc)
    accs.append(acc)
    f1s.append(f1_val)

print("\n==================================================")
print(f"Mean AUC: {np.mean(aucs):.4f} (+/- {np.std(aucs):.4f})")
print(f"Mean Accuracy: {np.mean(accs):.4f} (+/- {np.std(accs):.4f})")
print(f"Mean F1: {np.mean(f1s):.4f} (+/- {np.std(f1s):.4f})")



===== Fold 1 =====
0:	test: 0.8623894	best: 0.8623894 (0)	total: 6.02s	remaining: 3h 20m 43s
200:	test: 0.9202280	best: 0.9218907 (119)	total: 18m 6s	remaining: 2h 42m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9218907192
bestIteration = 119

Shrink model to first 120 iterations.
Best threshold = 0.48
AUC = 0.9219, Accuracy = 0.8515, F1 = 0.7976

===== Fold 2 =====
0:	test: 0.9516872	best: 0.9516872 (0)	total: 6.48s	remaining: 3h 36m 1s
200:	test: 0.9642958	best: 0.9642958 (200)	total: 17m 31s	remaining: 2h 36m 52s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9665231628
bestIteration = 202

Shrink model to first 203 iterations.
Best threshold = 0.59
AUC = 0.9665, Accuracy = 0.9328, F1 = 0.9045

===== Fold 3 =====
0:	test: 0.3585660	best: 0.3585660 (0)	total: 6.33s	remaining: 3h 31m 1s
200:	test: 0.9902525	best: 0.9902525 (200)	total: 17m 57s	remaining: 2h 40m 44s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9